In [86]:
import math # for sqrt...
import pandas as pd
import numpy as np
import matplotlib as mpl
import seaborn as sb
from sklearn.model_selection import train_test_split # for train test split
from sklearn.metrics import mutual_info_score # for mutual information score
from sklearn.metrics import accuracy_score # for accuracy score
from sklearn.metrics import mean_squared_error # for q6
from sklearn.feature_extraction import DictVectorizer # for one-hot encoding
from sklearn.linear_model import LogisticRegression # for log reg
from sklearn.linear_model import Ridge # for q6
import pickle

In [46]:
df = pd.read_csv("housing.csv")
df = df[['latitude',
         'longitude',
         'housing_median_age',
         'total_rooms',
         'total_bedrooms',
         'population',
         'households',
         'median_income',
         'median_house_value',
         'ocean_proximity']].copy()

In [47]:
df.isna().sum()[df.isna().sum()>0]

total_bedrooms    207
dtype: int64

In [48]:
# Data preparation

# Select only the features from above and fill in the missing values with 0
df['total_bedrooms'] = df['total_bedrooms'].fillna(0, inplace=False)


In [49]:
# Create a new column rooms_per_household by dividing the column total_rooms by the column households from dataframe.
df['rooms_per_household'] = df['total_rooms'] / df['households']
df[['total_rooms', 'households', 'rooms_per_household']].head()

,total_rooms,households,rooms_per_household
0,880.0,126.0,6.984127
1,7099.0,1138.0,6.238137
2,1467.0,177.0,8.288136
3,1274.0,219.0,5.817352
4,1627.0,259.0,6.281853


In [50]:
# Create a new column bedrooms_per_room by dividing the column total_bedrooms by the column total_rooms from dataframe.
df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']
df[['total_bedrooms', 'total_rooms', 'bedrooms_per_room']].head()

,total_bedrooms,total_rooms,bedrooms_per_room
0,129.0,880.0,0.146591
1,1106.0,7099.0,0.155797
2,190.0,1467.0,0.129516
3,235.0,1274.0,0.184458
4,280.0,1627.0,0.172096


In [51]:
# Create a new column population_per_household by dividing the column population by the column households from dataframe.
df['population_per_household'] = df['population'] / df['households']
df[['population', 'households', 'population_per_household']].head()

,population,households,population_per_household
0,322.0,126.0,2.555556
1,2401.0,1138.0,2.109842
2,496.0,177.0,2.802260
3,558.0,219.0,2.547945
4,565.0,259.0,2.181467


In [52]:
# Question 1
# What is the most frequent observation (mode) for the column ocean_proximity?
df['ocean_proximity'].value_counts()
# Options:
# NEAR BAY
# --> <1H OCEAN <--
# INLAND
# NEAR OCEAN

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [53]:
list(df.columns)

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value',
 'ocean_proximity',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [54]:
# Split the data
# Split your data in train/val/test sets, with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
# Make sure that the target value (median_house_value) is not in your dataframe.
X_train_valid, X_test, y_train_valid, y_test = train_test_split(
    df.drop('median_house_value', axis = 1),
    df[['median_house_value']],
    train_size = 0.8,
    test_size = 0.2,
    random_state = 42)

print(df.shape)
print()
print(X_train_valid.shape)
print(y_train_valid.shape)
print()
print(X_test.shape)
print(y_test.shape)

(20640, 13)

(16512, 12)
(16512, 1)

(4128, 12)
(4128, 1)


In [55]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_valid,
    y_train_valid,
    train_size = 0.75,
    test_size = 0.25,
    shuffle = False)
print(X_train.shape)
print(y_train.shape)
print()
print(X_valid.shape)
print(y_valid.shape)

(12384, 12)
(12384, 1)

(4128, 12)
(4128, 1)


In [56]:
# Question 2
# Create the correlation matrix for the numerical features of your train dataset.
num_columns = list(X_train.dtypes[df.dtypes == 'float64'].index)
cat_columns = list(X_train.dtypes[df.dtypes == 'object'].index)
corr_matrix = X_train[num_columns].corr()
corr_matrix.abs().unstack().sort_values(ascending=False).drop_duplicates()
# In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
# What are the two features that have the biggest correlation in this dataset?
# Options:
# --> total_bedrooms and households <--
# total_bedrooms and total_rooms
# population and households
# population_per_household and total_rooms

latitude                  latitude                    1.000000
households                total_bedrooms              0.980122
total_rooms               total_bedrooms              0.932220
longitude                 latitude                    0.924692
households                total_rooms                 0.922343
population                households                  0.912609
total_bedrooms            population                  0.883679
total_rooms               population                  0.864225
bedrooms_per_room         median_income               0.612738
                          rooms_per_household         0.495944
rooms_per_household       median_income               0.390775
total_rooms               housing_median_age          0.357091
housing_median_age        total_bedrooms              0.316835
                          households                  0.299384
                          population                  0.295541
total_rooms               median_income               0

In [57]:
# Make median_house_value binary
# We need to turn the median_house_value variable from numeric into binary.
# Let's create a variable above_average which is 1 if the median_house_value is above its mean value and 0 otherwise.
print(y_train['median_house_value'].mean())
above_average_train = (y_train['median_house_value'] > y_train['median_house_value'].mean()).astype(int)
print(y_valid['median_house_value'].mean())
above_average_valid = (y_valid['median_house_value'] > y_valid['median_house_value'].mean()).astype(int)
print(y_test['median_house_value'].mean())
above_average_test = (y_test['median_house_value'] > y_test['median_house_value'].mean()).astype(int)

207197.73796834625
207185.56104651163
205500.30959302327


In [58]:
# Question 3
# Calculate the mutual information score with the (binarized) price 
# for the categorical variable that we have. Use the training set only.
# What is the value of mutual information?
mutual_info_score(above_average_train,X_train['ocean_proximity']).round(2)
# Round it to 2 decimal digits using round(score, 2)
# Options:
# 0.263
# 0.00001
# --> 0.101 <--
# 0.15555

0.1

In [60]:
X_train_columns = X_train.columns
X_train_columns

Index(['latitude', 'longitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity', 'rooms_per_household', 'bedrooms_per_room',
       'population_per_household'],
      dtype='object')

In [61]:
# Question 4
# Now let's train a logistic regression
# Remember that we have one categorical variable ocean_proximity in the data. 
# Include it using one-hot encoding.
train_dict = X_train[cat_columns + num_columns].to_dict(orient='records')
# print(train_dict)
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
# print(dv.get_feature_names_out())
X_train = dv.transform(train_dict)

valid_dict = X_valid[cat_columns + num_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(valid_dict)
X_valid = dv.transform(valid_dict)

test_dict = X_test[cat_columns + num_columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(test_dict)
X_test = dv.transform(test_dict)


In [62]:
# important, obvious-but-not-obvious intermediate step:
# logistic regression only makes sense for binary variable...
# binary variable here: 'above_average' in X!
# y_train_logreg = X_train

train_dict

[{'ocean_proximity': 'NEAR OCEAN',
  'latitude': 32.71,
  'longitude': -117.03,
  'housing_median_age': 33.0,
  'total_rooms': 3126.0,
  'total_bedrooms': 627.0,
  'population': 2300.0,
  'households': 623.0,
  'median_income': 3.2596,
  'rooms_per_household': 5.017656500802568,
  'bedrooms_per_room': 0.20057581573896352,
  'population_per_household': 3.691813804173355},
 {'ocean_proximity': 'NEAR OCEAN',
  'latitude': 33.77,
  'longitude': -118.16,
  'housing_median_age': 49.0,
  'total_rooms': 3382.0,
  'total_bedrooms': 787.0,
  'population': 1314.0,
  'households': 756.0,
  'median_income': 3.8125,
  'rooms_per_household': 4.473544973544974,
  'bedrooms_per_room': 0.23270254287403902,
  'population_per_household': 1.7380952380952381},
 {'ocean_proximity': 'NEAR OCEAN',
  'latitude': 34.66,
  'longitude': -120.48,
  'housing_median_age': 4.0,
  'total_rooms': 1897.0,
  'total_bedrooms': 331.0,
  'population': 915.0,
  'households': 336.0,
  'median_income': 4.1563,
  'rooms_per_hous

In [63]:
# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of Scikit-Learn, 
# fit the model with these parameters:
model = LogisticRegression(solver = "liblinear", 
                           C = 1.0, 
                           max_iter = 1000, 
                           random_state = 42)

#model.fit(X_train, 
#         y_train.values.ravel())
model.fit(X_train,
         above_average_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [64]:
#filename = 'hw3_logistic_regression.sav'

# pickle.dump(model, open(filename, 'wb'))

# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
# print(result)

In [65]:
above_average_valid_pred = model.predict(X_valid)

In [66]:
above_average_valid_pred_df = pd.DataFrame(above_average_valid_pred, columns = ['above_average'])

In [67]:
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
accuracy_score_full_model = accuracy_score(above_average_valid_pred, above_average_valid)
# Options:
# 0.60
# 0.72
# --> 0.84 <--
# 0.95
print(accuracy_score_full_model)

0.8270348837209303


In [68]:
# model.predict_proba(X_valid)

In [69]:
# Question 5
# Let's find the least useful feature using the feature elimination technique.
# Train a model with all these features (using the same parameters as in Q4).
# Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
df.columns#.get_loc('total_rooms')

Index(['latitude', 'longitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity', 'rooms_per_household',
       'bedrooms_per_room', 'population_per_household'],
      dtype='object')

In [70]:
# train without total_rooms
model_wo_total_rooms = LogisticRegression(solver = "liblinear", C = 1.0, max_iter = 1000, random_state = 42)
model_wo_total_rooms.fit(np.delete(X_train, 3, axis=1),
                         above_average_train)
above_average_valid_pred_wo_total_rooms = model_wo_total_rooms.predict(np.delete(X_valid, 3, axis=1))

accuracy_score_wo_total_rooms = accuracy_score(above_average_valid_pred_wo_total_rooms, above_average_valid)
print(accuracy_score_wo_total_rooms)

0.8287306201550387


In [71]:
# train without total_bedrooms
model_wo_total_bedrooms = LogisticRegression(solver = "liblinear", C = 1.0, max_iter = 1000, random_state = 42)
model_wo_total_bedrooms.fit(np.delete(X_train, 4, axis=1),
                         above_average_train)
above_average_valid_pred_wo_total_bedrooms = model_wo_total_bedrooms.predict(np.delete(X_valid, 4, axis=1))

accuracy_score_wo_total_bedrooms = accuracy_score(above_average_valid_pred_wo_total_bedrooms, above_average_valid)
print(accuracy_score_wo_total_bedrooms)

0.8287306201550387


In [72]:
# train without population
model_wo_population = LogisticRegression(solver = "liblinear", C = 1.0, max_iter = 1000, random_state = 42)
model_wo_population.fit(np.delete(X_train, 5, axis=1),
                         above_average_train)
above_average_valid_pred_wo_population = model_wo_population.predict(np.delete(X_valid, 5, axis=1))

accuracy_score_wo_population = accuracy_score(above_average_valid_pred_wo_population, above_average_valid)
print(accuracy_score_wo_population)

0.7749515503875969


In [73]:
# train without households
model_wo_households = LogisticRegression(solver = "liblinear", C = 1.0, max_iter = 1000, random_state = 42)
model_wo_households.fit(np.delete(X_train, 6, axis=1),
                         above_average_train)
above_average_valid_pred_wo_households = model_wo_households.predict(np.delete(X_valid, 6, axis=1))

accuracy_score_wo_households = accuracy_score(above_average_valid_pred_wo_households, above_average_valid)
print(accuracy_score_wo_households)

0.8275193798449613


In [77]:
# compare accuracies
print(abs(accuracy_score_wo_total_rooms - accuracy_score_full_model),
     '\n',
     abs(accuracy_score_wo_total_bedrooms - accuracy_score_full_model),
     '\n',
     abs(accuracy_score_wo_population - accuracy_score_full_model),
     '\n',
     abs(accuracy_score_wo_households - accuracy_score_full_model))

0.0016957364341084746 
 0.0016957364341084746 
 0.05208333333333337 
 0.0004844961240310086


In [ ]:
# Which of following feature has the smallest difference?
# total_rooms
# total_bedrooms
# population
# --> households <--
# note: the difference doesn't have to be positive

In [91]:
# Question 6
# For this question, we'll see how to use a linear regression model from Scikit-Learn
# We'll need to use the original column 'median_house_value'. Apply the logarithmic transformation to this column.
logy_train = np.log1p(y_train).copy() 
logy_valid = np.log1p(y_valid).copy()
logy_test = np.log1p(y_test).copy()
# Fit the Ridge regression model (model = Ridge(alpha=a, solver="sag", random_state=42)) on the training data.
# This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
model_ridge_a_0 = Ridge(alpha = 0, solver = "sag", random_state = 42)
model_ridge_a_0.fit(X_train, logy_train)
model_ridge_pred_valid_a_0 = model_ridge_a_0.predict(X_valid)
rmse_ridge_a_0 = math.sqrt(mean_squared_error(logy_valid, model_ridge_pred_valid_a_0))

model_ridge_a_0_01 = Ridge(alpha = 0.01, solver = "sag", random_state = 42)
model_ridge_a_0_01.fit(X_train, logy_train)
model_ridge_pred_valid_a_0_01 = model_ridge_a_0_01.predict(X_valid)
rmse_ridge_a_0_01 = math.sqrt(mean_squared_error(logy_valid, model_ridge_pred_valid_a_0_01))

model_ridge_a_0_1 = Ridge(alpha = 0.1, solver = "sag", random_state = 42)
model_ridge_a_0_1.fit(X_train, logy_train)
model_ridge_pred_valid_a_0_1 = model_ridge_a_0_1.predict(X_valid)
rmse_ridge_a_0_1 = math.sqrt(mean_squared_error(logy_valid, model_ridge_pred_valid_a_0_1))

model_ridge_a_1 = Ridge(alpha = 1, solver = "sag", random_state = 42)
model_ridge_a_1.fit(X_train, logy_train)
model_ridge_pred_valid_a_1 = model_ridge_a_1.predict(X_valid)
rmse_ridge_a_1 = math.sqrt(mean_squared_error(logy_valid, model_ridge_pred_valid_a_1))

model_ridge_a_10 = Ridge(alpha = 10, solver = "sag", random_state = 42)
model_ridge_a_10.fit(X_train, logy_train)
model_ridge_pred_valid_a_10 = model_ridge_a_10.predict(X_valid)
rmse_ridge_a_10 = math.sqrt(mean_squared_error(logy_valid, model_ridge_pred_valid_a_10))

print('rmse ridge a=0: ', rmse_ridge_a_0,
      '\n',
      'rmse ridge a=0.01: ', rmse_ridge_a_0_01,
      '\n',
      'rmse ridge a=0.1: ', rmse_ridge_a_0_1,
      '\n',
      'rmse ridge a=1: ', rmse_ridge_a_1,
      '\n',
      'rmse ridge a=10: ', rmse_ridge_a_10)
      
# Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.
# If there are multiple options, select the smallest alpha.
# Options:
# --> 0 <--
# 0.01
# 0.1
# 1
# 10

rmse ridge a=0:  0.5340383274771877 
 rmse ridge a=0.01:  0.5340383274949412 
 rmse ridge a=0.1:  0.5340383276680537 
 rmse ridge a=1:  0.534038329372536 
 rmse ridge a=10:  0.5340383464439613
